In [84]:
import time
import sys
import codecs
import re
import pdb # ajouter pdb.set_trace() à l'endroit où on veut le débugueur

#Paramétrage

- ajouter le nom de votre groupe dans *nomGroupe*
- ajouter le nom de votre fichier corpus dans *nomCorpus*

In [85]:
#nomGroupe="GBCM"
#nomCorpus="BouvardPecuchet-Ch1"
nomGroupe="GGM"
nomCorpus="scolaire"


prefixe=nomGroupe+"-"
fichierLexique="../../phonemisation/bdlexique.txt"
lexicon=codecs.open(fichierLexique,"r","utf8")
#lexicon=codecs.open(fichierLexique,"r")

fichierCorpus=prefixe+nomCorpus+".txt"
fichier=codecs.open(fichierCorpus,"r","utf8")
#fichier=codecs.open(fichierCorpus,"r")

fichierExceptions=prefixe+"inconnus.txt"
try:
    exceptions=codecs.open(fichierExceptions,"r","utf8")
    #exceptions=codecs.open(fichierExceptions,"r")
    fichier_exceptions=True
except:
    fichier_exceptions=False

In [86]:
facultatives = 0

In [87]:
phrases=[]
phon={}
result=[]
nouvellesExceptions = []
output=[]

#Préparation des fichiers

###Modif GB 18/11/14

- gestion du passage en minuscules des majuscules accentuées

In [88]:
encodageUTF8=True
def lowerAccents(chaine):
    if isinstance(chaine,str):
        try:
            result=chaine.decode("utf8").lower()
        except:
            print chaine
        return result
    elif isinstance(chaine,unicode):
        result=chaine.lower()
    if not encodageUTF8:
        result=result.encode("utf8")
    return result

In [89]:
print (lowerAccents("Être"))

être


In [49]:
for line in fichier:
    line=line.strip()
    if line != '':
        phrases.append(line)

In [50]:
fichier.close()

#Lire BDLex
extraire de BDLex 0.forme fléchie, 1.phonétique, 2.liaison, 3.cat-gram, 4.genre+nombre

In [51]:
for entry in lexicon:
    entry=entry.strip()
    p=entry.split(';')
    phon[lowerAccents(p[0])]=(lowerAccents(p[0]),p[1],p[2],p[3],p[4])

In [52]:
lexicon.close()

#Lire les exceptions

extraire du fichier d'exceptions les mêmes données que pour BDLex

###Modif GB 12/04/14
- fait une liste des exceptions lues pour ne pas les rajouter à la fin
- éviter de tenir compte des exceptions non renseignées
 - les mots du fichier exceptions sans transcriptions étaient transcrits par une chaine vide...

In [53]:
if fichier_exceptions:
    oldExceptions={}
    for entry in exceptions:
        entry=entry.strip()
        p=entry.split(';')
        if len(p[1])!=0:
            phon[lowerAccents(p[0])]=(lowerAccents(p[0]),p[1],p[2],p[3],p[4])
        oldExceptions[lowerAccents(p[0])]=entry
    exceptions.close()

## Fonctions

### 1. Vérifier si le mot existe

algorithme

+ si le mot est dans BDLex, ok
+ s'il y a un espace dans le mot,
    * le mot est divisé en deux et
    * si les mots existent dans bdlex, ok
    * sinon les mots sont ajoutés aux nouvelles exceptions et mis entre étoiles
+ s'il y a un apostrophe dans le mot,
    * le mot est divisé en deux
    * si les mots existent dans bdlex, ok
    * sinon les mots sont ajoutés aux nouvelles exceptions et mis entre étoiles
+ dans les autres cas, le mot est ajouté aux nouvelles exceptions et mis entre étoiles

In [54]:
def verifier_mot(mot):
        sampa=""
        if mot in phon.keys():
            sampa += phon[mot][0]
        elif " " in mot:
            mots = mot.split()
            for mot in mots:
                if mot in phon.keys():
                    sampa += phon[mot][0]+" "
                elif mot != "":
                    sampa += "***"+mot+"*** "
        elif u"'" in mot:
            mots = mot.split(u"'")
            mots[0]=mots[0]+"'"
            for mot in mots:
                if mot in phon.keys():
                    sampa += phon[mot][0]+" "
                elif mot != "":
                    nouvellesExceptions.append(mot)
                    sampa += "***"+mot+"*** "
        elif mot != "": 
            nouvellesExceptions.append(mot)
            sampa="***"+mot+"*** "
        return sampa

In [55]:
verifier_mot("manger")

u'manger'

In [56]:
verifier_mot("manger mt")

u'manger ***mt*** '

In [57]:
verifier_mot("j'ai")

u"j' ai "

In [58]:
verifier_mot("y'a")

u"y' a "

In [59]:
verifier_mot(u"«monsieur")

u'***\xabmonsieur*** '

### 2. traduire le SAMPA de BDLexique en API

#Modif GB 12/04/14
- ajout du r et du â
- ajout des exemples associés en dessous

In [60]:
# traduire SAMPA-BDLex en API
def sampa2api(sampa):
    api=re.sub(u'S',u'ʃ',sampa) 
    api=re.sub(u'Z',u'ʒ', api)
    api=re.sub(u'N',u'ŋ',api)
    api=re.sub(u'J',u'ɲ',api)
    api=re.sub(u'r',u'ʁ',api) 
    api=re.sub(u'H',u'ɥ',api)
    api=re.sub(u'E',u'ɛ',api)
    api=re.sub(u'2',u'ø',api)
    api=re.sub(u'9',u'œ',api)
    api=re.sub(u'6',u'ə',api)
    api=re.sub(u'O',u'ɔ',api)
    api=re.sub(u'è',u'e',api)   
    api=re.sub(u'ò',u'o',api)    
    api=re.sub(u'â',u'ɑ̃',api)   
    api=re.sub(u'ê',u'ɛ̃',api)   
    api=re.sub(u'û',u'œ̃',api)  
    api=re.sub(u'ô',u'ɔ̃',api)       
    api=re.sub(u'@',u'ə',api)
    api=re.sub(u'n"',u'n',api) 
    api=re.sub(u't"',u't',api) 
    api=re.sub(u'z"',u'z',api) 
    api=re.sub(u'R"',u'ʁ',api) 
    api=re.sub(u'p"',u'p',api) 
    return api

In [61]:
sampa2api(phon["ai"][1])

u'e'

In [62]:
sampa2api(phon["chat"][1])

u'\u0283a'

In [63]:
sampa2api(phon["aucun"][1])

u'ok\u0153\u0303'

In [64]:
sampa2api(phon["chant"][1])

u'\u0283\u0251\u0303'

In [65]:
sampa2api(phon[u"marché"][1])

u'ma\u0281\u0283e'

In [66]:
sampa2api(phon["monsieur"][1])

u'm\u0259sj\xf8'

### 3. trimer le mot en cours

algorithme

+ la ponctuation est remplacée par un espace
+ les espaces aux extrémités sont effacés
+ le mot est mis en minuscules

In [67]:
def trimer(mot):
    mot=lowerAccents(mot)
    for p in u',;.:-?!()“”‘’‛‟′″´˝"«»':
        mot=mot.replace(p, ' ')
    mot=mot.strip()
    return mot

In [68]:
trimer(u",;. :trois-?!( )")

u'trois'

In [69]:
trimer("(essai)chaud")

u'essai chaud'

In [70]:
trimer("Garçon, monsieur.")

u'gar\xe7on  monsieur'

In [71]:
print (trimer("PÊCHÉ"))

pêché


In [72]:
trimer("vont-ils")

u'vont ils'

In [73]:
trimer("j‘ai")

u'j ai'

In [74]:
trimer("«y’a»")

u'y a'

In [75]:
trimer(u"«monsieur,")

u'monsieur'

### 4. Vérifier si la liaison est possible

algorithme

+ si le mot courant et le suivant ne sont pas dans lexicon, pas de liaison
+ si le mot a une consonne dans le champ de la voyelle de liaison, check1 est vrai
+ si le mot suivant commence par une voyelle, check2 est vrai

  si check1 et check2 sont vrais, il y a liaison

In [76]:
def liaison_possible(phrase ,mot , mot_numero):
    check1=0
    check2=0
    if mot in phon and phrase[mot_numero+1] in phon:
        consonnes=['k"', '(kt)"', 'n"', 'p"', 'R"', '@t"', 't"', '-V', '+V', '@z"', 'z"']
        phoneme=phon[mot][2]
        for phoneme in consonnes:
            check1=1
        
        voyelles=[u"H", u"j", u"w", 
                  u"i"
                  u"y", 
                  u"u", u"û",  
                  u"a", u"â"
                  u"O", u"ò", u"o", u"ô", 
                  u"E", u"è", u"e", u"ê", 
                  u"2", u"9", u"6", u"@", 
                  ]
        mot_suivant=phon[phrase[mot_numero+1]][1]
        for v in voyelles:
            if mot_suivant.startswith(v):
                check2=1

    if check1 and check2 :
        return True
    else:
        return False

### 5. vérifier si la liaison est obligatoire

algorithme

+ si le mot courant et le suivant sont dans un des cas de figure, il y a liaison
+ sinon pas de liaison

In [77]:
def liaison_obligatoire(phrase, mot, mot_numero):
    determinant=["d", "P"]
    nom=["N", "G", "M"]
    adjectif=["J", "G", "M"]
    pronompers=["P"]
    verbe=["V"]
    catgram_mot1=phon[phrase[mot_numero]][3]
    catgram_mot2=phon[phrase[mot_numero+1]][3]

    if catgram_mot1 in determinant and catgram_mot2 in nom :
        return True

    elif catgram_mot1 in determinant and catgram_mot2 in adjectif :
        return True
 
    elif catgram_mot1 in pronompers and catgram_mot2 in verbe :
        return True

    elif catgram_mot1 in verbe and catgram_mot2 in pronompers :
        return True

    else:
        return False

In [78]:
phon["sommes"]

(u'sommes', u'sOm', u'@', u'V', u'2S')

In [79]:
phon["armes"]

(u'armes', u'arm', u'@', u'V', u'2S')

In [80]:
phon["cet"]

(u'cet', u'sEt', u'+V', u'd', u'MS')

In [81]:
phon["monsieur"]

(u'monsieur', u'm6sj2', u'', u'N', u'MS')

Cas de figure possibles:

- DET + N
    * ri + N:   d'animal, 
    * di + N:   certains éléphants
    * rd + N:   les animaux
    * dd + N:   ces étés, cet été
    * dp + N:   ton anorak
    * rc + N:   aux armes
- DET + ADJ:
    * ri + ADJ:   d'énormes
    * di + ADJ:   plusieurs immenses
    * rd + ADJ:   les immenses
    * dd + ADJ:   cet immense
    * dp + ADJ:   son immense
    * rc + ADJ:   aux immenses
- PERS + V:
    * SS + V:   m'épate
- V + PRO PERS: 
    * V + SS:   vont-ils


algorithme

+ si le mot courant et le suivant sont dans un des cas de figure, il y a liaison
+ sinon pas de liaison

In [82]:
# vérifier si la liaison est facultative
def liaison_facultative(phrase, mot, mot_numero):
    #pdb.set_trace()
    nom=["N", "G", "M"]
    pluriel=["MP", "FP"]
    adjectif=["J", "G", "M"]
    verbe=["V"]
    pronompers=["P"]
    adverbe=["A"]
    preposition=["p"]
    catgram_mot1=phon[phrase[mot_numero]][3]
    catgram_mot2=phon[phrase[mot_numero+1]][3]
    genre_mot1=phon[phrase[mot_numero]][4]
    
    if (catgram_mot1 in nom) and (phon[phrase[mot_numero]][4] in pluriel) and (catgram_mot2 in adjectif) : 
        return True

    elif (catgram_mot1 in verbe) and (catgram_mot2 not in pronompers):
        return True

    elif catgram_mot1 in adverbe :
        return True
    
    elif catgram_mot1 in preposition : 
        return True

    else :
        return False

Cas de figure possibles :

- N pl + ADJ: 
    * N + ADJ: monstres énormes 
    * G + ADJ: rivaux énormes
- VERBE + TOUT-SAUF-PRO-PERS:
    * V + N sont éléphants
    * V + G sommes abdicaires
    * V + V sommes assis
    * V + A sommes admirablement
    * V + p sommes autour de
    * V + di ont aucune
    * V + rc sommes au
- ADV + QQCH:
    * ADV + N vraiment abruti
    * ADV + G vraiment abandonné
    * ADV + V vraiment aimé
    * ADV + J vraiment étonnant
    * ADV + ss vraiment ils
    * ADV + A vraiment étonnamment
    * ADV + p vraiment attendu
    * ADV + di vraiment autre 
    * ADV + rc vraiment au
- PREP + QQCH:
    * PREP + N très amoureux
    * PREP + G très abandonné
    * PREP + V très aimé
    * PREP + J très étonnant
    * PREP + SS très ils
    * PREP + A très étonnamment
    * PREP + p très attendu
    * PREP + di très autre
    * PREP + rc très au


## Traitement

+ Partie 1
*chaque phrase est prise individuellement,
    * découpée en blocs,
        * qui sont chacuns trimés si ce sont des mots
        * s'il y a plusieurs mots dans le bloc, ils sont séparés
    + Partie 2
    * pour chaque couple de mots
        * si la liaison est possible,
            * et qu'elle est obligatoire, l'api avec la liaison est généré
            * et qu'elle est facultative,
                * si l'utilisateur l'a choisi, l'api avec la liaison est généré
                * sinon l'api sans la liaison est généré

        + Partie 3
        * si la liaison n'est pas possible,
            * si le mot est dans bdlex, l'api est généré
            * sinon le mot est laissé tel quel (il a déjà les étoiles)        

    * pour le dernier mot de la phrase, 
        * si le mot est dans bdlex, l'api est généré
        * sinon le mot est laissé tel quel (il a déjà les étoiles) 

+ Partie 4
* le message à l'utilisateur et la phrase en api est imprimée

#Modif GB 12/04/14
- suppression du délai dans la boucle
 - pour 1500 lignes => 3 secondes sans ralentisseur, 1503 secondes avec 

In [83]:
# partie 1
a=1
for phrase in phrases:
    api = ""
    mots=re.split(u"[->< /@\_]", phrase)
    phrasePropre = u""
    for mot in mots:
        mot = trimer(mot)
        mot = verifier_mot(mot)
        phrasePropre += mot+u" "
    phraseMots = phrasePropre.strip()
    phraseMots = phrasePropre.split()

    # partie 2    
    mot_numero=0
    while mot_numero <= len(phraseMots)-2:
        
        if liaison_possible(phraseMots, phraseMots[mot_numero], mot_numero):
 
            if liaison_obligatoire(phraseMots, phraseMots[mot_numero], mot_numero):
                api += sampa2api(phon[phraseMots[mot_numero]][1])+sampa2api(phon[phraseMots[mot_numero]][2]+" ")

            elif liaison_facultative(phraseMots, phraseMots[mot_numero], mot_numero):
                if facultatives:
                    api += sampa2api(phon[phraseMots[mot_numero]][1])+sampa2api(phon[phraseMots[mot_numero]][2]+" ")
                else :
                    api += sampa2api(phon[phraseMots[mot_numero]][1])+" "
            else:
                if phraseMots[mot_numero] in phon:
                    api += sampa2api(phon[phraseMots[mot_numero]][1]+" ")
                else:
                    api += phraseMots[mot_numero]+" "
        # partie 3
        else:
            if phraseMots[mot_numero] in phon:
                api += sampa2api(phon[phraseMots[mot_numero]][1]+" ")
            else:
                api += phraseMots[mot_numero]+" "
        mot_numero = mot_numero+1
    
    if phraseMots[len(phraseMots)-1] in phon:
        api += sampa2api(phon[phraseMots[len(phraseMots)-1]][1])
    else:
        api += phraseMots[mot_numero]
    
    # partie 4
    sys.stdout.write("Traitement de "+str(a)+" sur "+ str(len(phrases))+ " phrases. "+"\n"+phrase+"\n"+api+"\n\n"+chr(20))
    sys.stdout.flush()
    #time.sleep(1)
    a=a+1
    output.append(api)

Traitement de 1 sur 2552 phrases. 
on a travaillé
ɔ̃n a tʁavaje

Traitement de 2 sur 2552 phrases. 
un petit peu  hein   euh  pas encore on commence juste notre travail
œ̃ pəti pø ɛ̃ ə pa ɑ̃kɔʁ ɔ̃ komɑ̃s ʒyst nɔtʁ tʁavaj

Traitement de 3 sur 2552 phrases. 
euh  on est allés voir une exposition
ə ɔ̃n e ale vwaʁ ynə ɛkspozisjɔ̃

Traitement de 4 sur 2552 phrases. 
et aujourd'hui on doit regarder
e oʒuʁdɥi ɔ̃ dwa ʁəgaʁde

Traitement de 5 sur 2552 phrases. 
des peintures ou bien des sculptures
de pɛ̃tyʁ u bjɛ̃ de skyltyʁ

Traitement de 6 sur 2552 phrases. 
de
d

Traitement de 7 sur 2552 phrases. 
alors qu'est ce que c'est d'arbre
alɔʁ k e s k s e d aʁbʁ

Traitement de 8 sur 2552 phrases. 
d'arbre
d aʁbʁ

Traitement de 9 sur 2552 phrases. 
d'arbre
d aʁbʁ

Traitement de 10 sur 2552 phrases. 
bon on a travaillé sur les arbres
bɔ̃ ɔ̃n a tʁavaje syʁ lez aʁbʁ

Traitement de 11 sur 2552 phrases. 
alors vous allez décrire
alɔʁ vuz ale dekʁiʁ

Traitement de 12 sur 2552 phrases. 
ce que vo

KeyboardInterrupt: 

#Modif GB 12/04/14
- Insertion d'un set sur les nouvellesExceptions pour éviter les entrées multiples
- Ajout d'un test pour vérifier que les nouvellesExceptions sont nouvelles

#TO DO
- Ajouter un message pour dire que le résultat a été concaténé au fichier existant si c'est le cas.

In [ ]:
#1.2.a. mettre la liste des inconnus dans le fichier inconnus.txt
with codecs.open(prefixe+"inconnus.txt", "w", "utf8") as f:
    for ligne in sorted(oldExceptions, key=lambda key: oldExceptions[key]):
        f.write(oldExceptions[ligne])
        f.write("\n")        
    for n in set(nouvellesExceptions):
        if not (n in oldExceptions.keys()): 
            f.write(n+";;;;;")
            f.write("\n")
        
#1.2.b. mettre les phrases phonémisées dans un fichier
with codecs.open(prefixe+"phonemise.txt", "w", "utf8") as f:
    for o in output:
        f.write(o)
        f.write("\n")

with codecs.open(prefixe+"verification.txt", "w", "utf8") as f:
    for a in range(len(output)-2):
        f.write(phrases[a]+"\n"+output[a])
        f.write("\n\n")